In [2]:
import imageio.v3 as iio

class MRIStudy:
    _early_post_contrast_num = -1
    _late_post_contrast_num = -1
    _number_of_slice = -1 
    _dicom_image = {} 
    _temporal_resolution = -1
    _early_diff_mm = -1 
    _early_diff_ss = -1 
    _late_diff_mm = -1
    _late_diff_ss = -1
    _base_dicom_folder = ""

    def __init__(self, base_path, timing_file):
        timing_dicom_data = open(f"{base_path}{timing_file}")
        for x in timing_dicom_data.readlines():
            if "earlyPostContrastNum" in x:
                self._early_post_contrast_num = int(x.split(':')[1])
            if "latePostContrastNum" in x:
                self._late_post_contrast_num = int(x.split(':')[1])
            if "earlydiffmm" in x: 
                self._early_diff_mm = int(x.split(':')[1])
            if "earlydiffss" in x:
                self._early_diff_ss = int(x.split(':')[1])
            if "latediffmm" in x:
                self._late_diff_mm = int(x.split(':')[1])
            if "latediffss" in x:
                self._late_diff_ss = int(x.split(':')[1])
            if "tempres" in x:
                self._temporal_resolution = float(x.split(':')[1])
            if "nslice" in x:
                self._number_of_slice = int(x.split(':')[1])
            if "dce_folders" in x:
                expression = "'"
                self._base_dicom_folder = f"{base_path}{x.split(expression)[1]}"
            if "fsort" in x: 
                from ast import literal_eval
                dce_stages = literal_eval(x.split(':')[1])
                for index, stage in enumerate(dce_stages):
                    self._dicom_image[index] = stage

    def dump(self): 
        print(f"{self._early_post_contrast_num}")
        print(f"{self._late_post_contrast_num}")
        print(f"{self._number_of_slice}")
        print(f"{self._temporal_resolution}")
        print(f"{self._early_diff_mm}")
        print(f"{self._early_diff_ss}")
        print(f"{self._late_diff_mm}")
        print(f"{self._late_diff_ss}")
        print(f"{self._base_dicom_folder}")
        for stage,value in self._dicom_image.items():
            print(f"Stage {stage} | Total {len(value)}: {value}")

    def pre_contrast_images(self) -> (int,list):
        pre_contrast_images = []
        for stage, images in self._dicom_image.items():
            if stage == 0:
                for image in images:
                    path = f"{self._base_dicom_folder}/{image}"
                    pre_contrast_images.append(iio.imread(path, plugin='DICOM'))
            
        return len(pre_contrast_images), pre_contrast_images
    
    def early_post_contrast_images(self) -> (int,list):
        early_post_contrast_images = []
        for stage, images in self._dicom_image.items():
            if stage == self._early_post_contrast_num:
                for image in images:
                    path = f"{self._base_dicom_folder}/{image}"
                    early_post_contrast_images.append(iio.imread(path, plugin='DICOM'))
                
        return len(early_post_contrast_images), early_post_contrast_images
    
    def late_post_contrast_images(self) -> (int,list):
        late_post_contrast_images = []
        for stage, images in self._dicom_image.items():
            if stage == self._late_post_contrast_num:
                for image in images:
                    path = f"{self._base_dicom_folder}/{image}"
                    late_post_contrast_images.append(iio.imread(path, plugin='DICOM'))
                
        return len(late_post_contrast_images), late_post_contrast_images
    
    def contrast_timing(self) -> dict:
        return { 
            "early_diff" : f"{self._early_diff_mm}:{self._early_diff_ss}",
            "late_diff" : f"{self._late_diff_mm}:{self._late_diff_ss}"
        }
    def temporal_resolution(self) -> float: 
        return self._temporal_resolution

In [3]:
study1 = MRIStudy("/workspaces/dcemri-ftv/data/manifest-1702186450548/ACRIN-6698/ACRIN-6698-169600/01-04-2001-169600T0-ACRIN-6698ISPY2MRIT0-82967/","timing_dicom_data.txt")

pci_count, pre_contrast_images = study1.pre_contrast_images()
epci_count, early_post_contrast_images = study1.early_post_contrast_images()
lpci_count, late_post_contrast_images = study1.late_post_contrast_images()

ct = study1.contrast_timing()
tr = study1.temporal_resolution()

print(pci_count)
print(epci_count)
print(lpci_count)
print(ct)
print(tr)
study1.dump()


Reading DICOM (examining files): 1/960 files (0.1%)

175/960 files (18.2%342/960 files (35.6%499/960 files (52.0%679/960 files (70.7%859/960 files (89.5%960/960 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/960 files (0.1218/960 files (22.7%452/960 files (47.1%683/960 files (71.1%918/960 files (95.6%960/960 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/960 files (0.1234/960 files (24.4%469/960 files (48.9%701/960 files (73.0%936/960 files (97.5%960/960 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/960 files (0.1229/960 files (23.9%346/960 files (36.0%589/960 files (61.4%832/960 files (86.7%960/960 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/960 files (0.1234/960 files (24.4%473/960 files (49.3%705/960 files (73.4%941/960 files (98.0%960/960 files (100.0%)
  Found 1 correct series.
Reading DICOM (examining files): 1/960 files (0.1231/960 files (24.1%472/960 files (49.2%710/960 files (74.0%950/960 files (99.0%960/960 fi

In [31]:
print("2d length")
print(len(pre_contrast_images[0]))
print(len(pre_contrast_images[0][0]))
print("2d 384*384")
#print(pre_contrast_images[0][380])
#print(pre_contrast_images[0][380][2])

for i in range(0,384):
    row = "|"
    for j in range(0, 384):
        formatted_val = str(pre_contrast_images[0][i][j])
        if len(formatted_val) == 1:
           formatted_val = f"00{formatted_val}"
        if len(formatted_val) == 2:
            formatted_val = f"0{formatted_val}"
        
        row += f" {formatted_val} |"
    print(row)

2d length
384
384
2d 384*384
| 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000 | 000

In [32]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

#print(images)
#plt.imshow(images[1], cmap='gray')

@widgets.interact(axial_slice1=(0,len(pre_contrast_images)-1))
def axial_slicer1(axial_slice1=0):
  fig, ax = plt.subplots(1, 1, figsize=(8, 8))
  ax.imshow(pre_contrast_images[axial_slice1]) #cmap='gray', )#aspect=axial_asp)
  ax.axis('off')

@widgets.interact(axial_slice2=(0,len(early_post_contrast_images)-1))
def axial_slicer2(axial_slice2=0):
  fig, ax = plt.subplots(1, 1, figsize=(8, 8))
  ax.imshow(early_post_contrast_images[axial_slice2])#, cmap='gray', )#aspect=axial_asp)
  ax.axis('off')

@widgets.interact(axial_slice3=(0,len(late_post_contrast_images)-1))
def axial_slicer3(axial_slice3=0):
  fig, ax = plt.subplots(1, 1, figsize=(8, 8))
  ax.imshow(late_post_contrast_images[axial_slice3])#, cmap='gray', )#aspect=axial_asp)
  ax.axis('off')

interactive(children=(IntSlider(value=0, description='axial_slice1', max=159), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=0, description='axial_slice2', max=159), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=0, description='axial_slice3', max=159), Output()), _dom_classes=('widge…